In [1]:
import cudf
import dask_cudf

In [2]:
import dask_cudf

In [3]:
from dask_cuml.linear_regression import LinearRegression

In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [4]:
# from dask.distributed import Client
# from dask_cuda import LocalCUDACluster
# cluster = LocalCUDACluster(threads_per_worker = 10)


In [5]:
from dask.distributed import Client
client = Client(scheduler_file = "../cluster.json")
client

Client Scheduler: tcp://10.2.169.115:8786 Dashboard: http://10.2.169.115:8787/status,Cluster Workers: 2 Cores: 2 Memory: 3.15 GB


In [6]:
import numba.cuda

devs = [i.id for i in numba.cuda.cudadrv.devices.gpus]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [7]:
def set_visible(i, n):
    import os, numba.cuda
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
    numba.cuda.close()
    numba.cuda.select_device(i)
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd

dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [8]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])


In [9]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

In [10]:
import numba.cuda
import cuml
def print_device(arr):
    print(str(numba.cuda.get_current_device()))
    print(str(cuml.device_of_ptr(arr.as_gpu_matrix(order="F"))))
    
[client.submit(print_device, part) for part in X_df.to_delayed()]

[<Future: status: pending, key: print_device-93462cf168bba9143ac4641eddd44d4e>,
 <Future: status: pending, key: print_device-067107fa9678bfa06740566f4fe7c5eb>,
 <Future: status: pending, key: print_device-1b11ba6cdde0fcb50d8454a67ead2658>,
 <Future: status: pending, key: print_device-4d0d0b0012ebcda2ce408a2fb4d12eea>]

In [11]:
client.who_has()

{"('from_cudf-035212611cc64ebf9253fd49ae78355e', 0)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 1)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 2)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 3)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 0)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 1)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 2)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 3)": (),
 'print_device-067107fa9678bfa06740566f4fe7c5eb': (),
 'print_device-1b11ba6cdde0fcb50d8454a67ead2658': (),
 'print_device-4d0d0b0012ebcda2ce408a2fb4d12eea': (),
 'print_device-93462cf168bba9143ac4641eddd44d4e': (),
 'set_visible-b2cc47a71d9c1bf2a02b4ece69564212': ('tcp://10.2.169.115:43755',),
 'set_visible-f448d1672c3bacc20a17c17b73d73424': ('tcp://10.2.169.115:37840',)}

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [12]:
client.who_has()

{"('from_cudf-035212611cc64ebf9253fd49ae78355e', 0)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 1)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 2)": (),
 "('from_cudf-035212611cc64ebf9253fd49ae78355e', 3)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 0)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 1)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 2)": (),
 "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 3)": (),
 'print_device-067107fa9678bfa06740566f4fe7c5eb': (),
 'print_device-1b11ba6cdde0fcb50d8454a67ead2658': (),
 'print_device-4d0d0b0012ebcda2ce408a2fb4d12eea': (),
 'print_device-93462cf168bba9143ac4641eddd44d4e': (),
 'set_visible-b2cc47a71d9c1bf2a02b4ece69564212': ('tcp://10.2.169.115:43755',),
 'set_visible-f448d1672c3bacc20a17c17b73d73424': ('tcp://10.2.169.115:37840',)}

In [13]:
lr = LinearRegression()

In [14]:
res = lr.fit(X_df, y_df)

{'tuple-338b9c22-58dc-46d6-bf37-e4936475b164': ('tcp://10.2.169.115:43755',), "('from_cudf-035212611cc64ebf9253fd49ae78355e', 3)": ('tcp://10.2.169.115:43755',), "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 3)": ('tcp://10.2.169.115:43755',), 'set_visible-b2cc47a71d9c1bf2a02b4ece69564212': ('tcp://10.2.169.115:43755',), "('from_cudf-035212611cc64ebf9253fd49ae78355e', 1)": ('tcp://10.2.169.115:43755',), 'print_device-067107fa9678bfa06740566f4fe7c5eb': ('tcp://10.2.169.115:43755',), "('from_cudf-035212611cc64ebf9253fd49ae78355e', 0)": ('tcp://10.2.169.115:43755',), 'set_visible-f448d1672c3bacc20a17c17b73d73424': ('tcp://10.2.169.115:37840',), 'print_device-1b11ba6cdde0fcb50d8454a67ead2658': ('tcp://10.2.169.115:37840',), "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 2)": ('tcp://10.2.169.115:43755',), "('from_cudf-ad5d5cc35a324c7fae97438d7b8659dd', 0)": ('tcp://10.2.169.115:43755',), 'print_device-93462cf168bba9143ac4641eddd44d4e': ('tcp://10.2.169.115:37840',), 'print_device-4d0d0b

res

In [15]:
lr.coeffs

[(('10.2.169.115', 43755),
  <Future: status: finished, type: DataFrame, key: get_result-3a30958f77621e3da821390de5ae9fa8>)]

In [16]:
g = lr.predict(X_df)

{'get_result-3a30958f77621e3da821390de5ae9fa8': <Future: status: finished, type: DataFrame, key: get_result-3a30958f77621e3da821390de5ae9fa8>}
EXEC_NODE: ('10.2.169.115', 43755)
ON WORKER: 1
NOT ON WORKER: 0
RES: [<Future: status: finished, type: NoneType, key: _predict_on_worker-196b1a109444b12f00bf52b0153dfc74>]


In [17]:
g

tornado.gen.Return([<Future: status: error, key: get_result-4cb9f3c25addc6b94499712fc253e2cc>])